# Creating the Commitment Transaction with in-flight htlc

In this section, we'll build a lightning channel commitment transaction with in-flight htlc from scratch using Python. We'll walk through each part of the transaction — how it's constructed and signed. The process will be tested using Bitcoin Core in regtest mode.

## Setup

For this notebook, we’ll use the basepoints derivated in the `chapter 0 - lightning node keys derivation` the funding transaction created in the `chapter 1 - channel funding transaction` and parts of the first commitment transaction created in the `chapter 2 - initial commitment transaction`.

In [1]:
# run notebook

%run "/home/pins-dev/Projects/Taproot-Lightning-Channels-Workshop/chapter 2 - initial commitment transaction/initial commitment transaction.ipynb"

2025-09-25T19:53:55.616000Z TestFramework (INFO): PRNG seed is: 6183810789571658583
2025-09-25T19:53:55.617000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_vlvpye9s
Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Payment BasePoint:  025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Bob Payment BasePoint:  02f98efd3f2b2fbe7bd83c419f5f64f8280798b8a9175fdb77c0091bbb95c79506
To obscure commitment number 0xb433fd43a66f
Alice funding pubkey: cd17635b908b952ca4cdaadc273a7795c929922c39ccc2677b5089274fe7f3c1
Alice funding privkey: e0c721280faae50c905e296c3e02e5eda49b42d5c5eb39024f411a54b0a7063d
Alice funding address: bcrt1pe5tkxkus3w2jefxd4twzwwnhjhyjny3v88xvyemm2zyjwnl870qsyeeqxk
Bob funding pubkey: 48e4418d18e9188008f98592bdc4a4c6d71261b11e07262a184014e8aa0a8d19
Bob funding privkey: 3acd5bf43be33ca167b2992870efb

aggregated Schnorr verifies? True
signed tx:  0200000000010135b5866958962ba53a34d3368ee371aee3f99078629f1c113109925daf4c81760000000000fd33b480024a01000000000000225120d6363615b3d00361158c0f48a4ef81ea12e214e8d56e24098759f6b4267dca8bdd3f0f0000000000225120a52421c04d821c008f15346cb655de2b7be6934ed692b4966e2d9ac13e2a8ebf014006a8187f0d148034b00d3221e5963aeabd531e461ad24d6ea69aa4f8f3faabee1cdb9164213f64d70bed22b57f6cf2d41ab003e2f808fb83210e9f02f5354fc16fa64320
{
  "txid": "6419ff655966cca525325f11c466391b8e9a743008cb6d4c1ba8bd357e07bcdf",
  "hash": "c5b893b3c65f64ad1d7381f7c3bf38f2245150773a70d6d57f4eca95be95f558",
  "version": 2,
  "size": 205,
  "vsize": 154,
  "weight": 616,
  "locktime": 541304431,
  "vin": [
    {
      "txid": "76814caf5d920931111c9f627890f9e3ae71e38e36d3343aa52b96586986b535",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "06a8187f0d148034b00d3221e5963aeabd531e461ad24d6ea69aa4f8f3faabee1cdb9164213f64d70

## HTLC (Hashed Time Locked Contracts)

Once the channel is established, it can be used to make payments via HTLCs (Hashed Time-Locked Contracts).

Updates to the channel state are sent in batches: one or more `update_ messages` are exchanged before a `commitment_signed` message is sent. In the diagram below, we show only one`update_message` per `commitment_signed` for simplicity:

    +-----------+                                                  +-----------+
    |           |------------------ update_add_htlc -------------->|           |
    |           |----- commitment_signed (partial sig & nonce) --->|           |
    |   Alice   |                                                  |    Bob    |
    |           |<------------- revoke_and_ack (nonce) ------------|           |
    |           |<---- commitment_signed  (partial sig & nonce) ---|           |
    |           |-------------- revoke_and_ack (nonce) ----------->|           |
    |           |                                                  |           |
    +-----------+                                                  +-----------+

The messages are defined in the [BOLT 2](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#adding-an-htlc-update_add_htlc).

### The `update_add_htlc` Message

Alice sends the `update_add_htlc`  message to Bob, which includes the details of the HTLC being offered:
- channel_id
- id (htlc counter starting in zero)
- amount_msat
- payment_hash
- cltv_expiry
- onion_routing_packet

### The `commitment_signed` Message

Shortly after sending the `update_add_htlc` message, Alice commits to the new channel state so that the HTLC can be safely included by Bob. At this point, Bob has the HTLC information and has constructed a new commitment transaction, but he hasn't yet received Alice's signature for it.

Alice sends a `commitment_signed`message to Bob, which includes her signature for the new commitment transaction as well as the HTLC output it contains:
- channel_id
- signature
- num_htlcs
- partial_signature_with_nonce

### The `revoke_and_ack` Message

Now that Bob has a new signed commitment, he needs to acknowledge it and revoke the previous one. He does this by sending a`revoke_and_ack` message to Alice:

- channel_id
- per_commitment_secret
- next_per_commitment_point
- nonce

Finally, Bob sends a `commitment_signed` message to Alice, and Alice responds with a `revoke_and_ack` message. At this point, both parties have fully signed the new commitment transaction.


### The Unsigned Transaction

### The Input

The input is the channel funding transaction.

In [ ]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(funding_channel_txid))[::-1]
funding_channel_index = 0
index = funding_channel_index.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# sequence: upper 8 bits are 0x80, lower 24 bits are the upper 24 bits of the obscured commitment number
# Commitment number of the second commitment transaction.
commitment_number = 1
# obscured commitment number is result of xor operation 
commitment_number_obscured = lower48_to_obscure ^ commitment_number
# Extract the upper 24 bits of the obscured commitment number
upper_24_bits = (commitment_number_obscured >> 24) & 0xFFFFFF
# Combine the upper 8 bits (0x80) with the lower 24 bits (upper 24 of obscured number)
sequence = (0x80 << 24) | upper_24_bits
# Convert to bytes (byte, big-endian)
sequence = sequence.to_bytes(4, byteorder='big')
# Convert to little-endian 
sequence = sequence[::-1]

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

print("Commitment Number Osbscured", hex(commitment_number_obscured))

### The Outputs

Let’s assume Alice is sending 0.5 million satoshis to Bob, for this reason Alice node is offering an HTLC to Bob's node of this value.

#### The Offered HTLC otuput

We will also add an offered HTLC otuput, which pays out to the remote party if they reveal the pre-image to a given hash before a certain CLTV timeout. After the timeout, the local party will be able to claim the output via the htlc-timeout transaction. If the commitment transaction is a revoked state, then the remote party should be able to sweep the output at any time.



    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +-------------------+
                    ---| P(revocation) + T |
                       +-------------------+
                                         ^
                                         |
                                   +-----------+        
                                   | T = t * G |
                                   +-----------+        
                                         ^
                                         |
     +---+   +-------------------------------------------------------+
     | t | = | TaggedHash ("Taptweak", P(revocation) || script_root) |
     +---+   +-------------------------------------------------------+
                                                             ^
                                                             |
                                                          +-----+
                          ------------------------------> | hAB |<------------------
                         |                                +-----+                   |
                         |                                                          |
                         |                                                          |
                      +----+                                                     +----+
                      | hA |                                                     | hB |
                      +----+                                                     +----+
                         ^                                                          ^
                         |                                                          |
      +---------------------------------+                                           |
      | P(local_htlc) OP_CHECKSIGVERIFY |                                           |
      |                                 |                                           |
      | P(remote_htlc) OP_CHECKSIG      |                                           |
      +---------------------------------+                                           |
      +---------------------------------------------------------------------------------------------+ 
      | OP_SIZE 32 OP_EQUALVERIFY OP_HASH160 <RIPEMD160(payment_hash)> OP_EQUALVERIFY P(remote_htlc) |
      |                                                                                              |
      | OP_CHECKSIG OP_1 OP_CHECKSEQUENCEVERIFY OP_DROP                                              |
      +----------------------------------------------------------------------------------------------+

In [ ]:
# Create Alice per-commitment
alice_per_commitment = per_commitment(alice_per_commitment_seed, commitment_number)
# Create Bob per-commitment
bob_per_commitment = per_commitment(bob_per_commitment_seed, commitment_number)

# Create Alice Delayed Public Key
alice_delayed_pubkey = derivate_key(alice_node_seed, family=4).get_pubkey(alice_per_commitment.get_pub())
print(f"Alice Delayed PubKey: {alice_delayed_pubkey.get_bytes(bip340=False).hex()}")

# Create Bob Revocation Public Key
bob_revocation_pubkey = derivate_revocation_key(bob_node_seed).get_pubkey(alice_per_commitment.get_pub())
print(f"Bob Revocation PubKey: {bob_revocation_pubkey.get_bytes(bip340=False).hex()}")

# Create Alice HTLC Public Key
alice_htlc_pubkey = derivate_key(alice_node_seed, family=2).get_pubkey(alice_per_commitment.get_pub())
print(f"Alice HTLC PubKey: {alice_htlc_pubkey.get_bytes(bip340=False).hex()}")

# Create Bob HTLC Public Key
bob_htlc_pubkey = derivate_key(bob_node_seed, family=2).get_pubkey(bob_per_commitment.get_pub())
print(f"Bob HTLC PubKey: {bob_htlc_pubkey.get_bytes(bip340=False).hex()}")

# Outputs for Alice Second Commitment Transaction
# 0x03 outputs
output_count = bytes.fromhex("03")

# ANCHOR OUTPUT
# Anchor output amount
anchor_output_value_satoshis = 330
anchor_output_value = anchor_output_value_satoshis.to_bytes(8, byteorder="little", signed=False)

# OP_16 OP_CSV
script = CScript([OP_16, OP_CHECKSEQUENCEVERIFY])

# Method: ser_string(data) is a function which adds compactsize to input data.
hash_input = TAPSCRIPT_VER + ser_string(script)

# Anchor Output script_root
script_root = tagged_hash("TapBranch", hash_input)

# Anchor Output Tagged Hash
taptweak = tagged_hash("TapTweak", alice_delayed_pubkey.get_bytes() + script_root)
alice_delayed_pubkey_tweaked = alice_delayed_pubkey.tweak_add(taptweak)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
alice_anchor_spk = bytes.fromhex("51") + varint_len(alice_delayed_pubkey_tweaked.get_bytes()) + alice_delayed_pubkey.get_bytes()

# OFFERED HTLC OUTPUT
# HTLC output amount
htlc_output_value_satoshis = 500000
htlc_output_value = htlc_output_value_satoshis.to_bytes(8, byteorder="little", signed=False)

# Create the leaf scripts A and B
# Leaf script A: P(local_htlc) OP_CHECKSIGVERIFY P(remote_htlc) OP_CHECKSIG



### Fee Calculation

In [ ]:
# Second commitment expected weight 1124 + 172 * num-untrimmed-htlc-outputs
commitment_weight = 1124 + 172 * 1

# From open_channel and update_feerate messages (1 sat/vb = 250 sat/kw)
feerate_per_kw = 250

# Calculate the fee for the second commitment transaction
commitment_fee = int(commitment_weight * feerate_per_kw / 1000)